In [1]:
import polars as pl
import polars_unit as plu

quantity = pl.Struct(
    {
        "value": pl.Int64,
        "unit": pl.String,
    }
)


df = pl.DataFrame(
    {
        "a": pl.Series(
            [
                (-1, "m"),
                (2, "m"),
                (3, "m"),
                (4, "m"),
                (5, "m"),
            ],
            dtype=quantity,
        ),
        "b": pl.Series(
            [
                (1, "m"),
                (2, "m"),
                (3, "m"),
                (4, "m"),
                (5, "m"),
            ],
            dtype=quantity,
        ),
        "c": [1, 2, 3, 4, 5],
    }
)


df.with_columns(aa=plu.abs("a"))

a,b,c,aa
struct[2],struct[2],i64,struct[2]
"{-1,""m""}","{1,""m""}",1,"{1,""m""}"
"{2,""m""}","{2,""m""}",2,"{2,""m""}"
"{3,""m""}","{3,""m""}",3,"{3,""m""}"
"{4,""m""}","{4,""m""}",4,"{4,""m""}"
"{5,""m""}","{5,""m""}",5,"{5,""m""}"


Numeric cast in polars

In [2]:
start = pl.DataFrame(
    {
        "int": [1, 2, 3],
        "float": [1.0, 2.0, 3.0],
    }
)
sum = start.with_columns(sum=pl.col("int") + pl.col("float"))
start.dtypes, sum.dtypes

([Int64, Float64], [Int64, Float64, Float64])

In [3]:
unit_dtype = pl.Struct(
    {
        "value": pl.Float64,
        "unit": pl.String,
    }
)


unit_op = plu.abs
polars_op = lambda x: x.abs()

df = pl.DataFrame(
    {
        "unit": pl.Series(
            [
                (-1, "m"),
                (0, "m"),
                (1, "m"),
                (2, "m"),
            ],
            dtype=unit_dtype,
        ),
    }
)
(
    df.with_columns(unit_op=unit_op("unit")).with_columns(
        polars=polars_op(pl.col("unit").struct.field("value"))
    )  # .all())["equal"]
    # .item()
)

unit,unit_op,polars
struct[2],struct[2],f64
"{-1.0,""m""}","{1.0,""m""}",1.0
"{0.0,""m""}","{0.0,""m""}",0.0
"{1.0,""m""}","{1.0,""m""}",1.0
"{2.0,""m""}","{2.0,""m""}",2.0


In [4]:
df = df.with_columns(unit_noop=plu.noop("unit")).with_columns(
    equal=(pl.col("unit_noop") == pl.col("unit"))
)
df

unit,unit_noop,equal
struct[2],struct[2],bool
"{-1.0,""m""}","{-1.0,""m""}",true
"{0.0,""m""}","{0.0,""m""}",true
"{1.0,""m""}","{1.0,""m""}",true
"{2.0,""m""}","{2.0,""m""}",true


In [5]:
df.dtypes

[Struct({'value': Float64, 'unit': String}),
 Struct({'value': Float64, 'unit': String}),
 Boolean]

In [6]:
df.with_columns(plu.add("unit", "unit"))

unit,unit_noop,equal
struct[2],struct[2],bool
"{-2.0,""m""}","{-1.0,""m""}",true
"{0.0,""m""}","{0.0,""m""}",true
"{2.0,""m""}","{1.0,""m""}",true
"{4.0,""m""}","{2.0,""m""}",true


In [7]:
series = pl.Series([1,2,3,4,5])
dtype = pl.Struct({"value": series.dtype, "unit": pl.String})

In [8]:
pl.struct(value = series, unit =  pl.lit("hello"), eager=True).dtype

Struct({'value': Int64, 'unit': String})

In [16]:
df = pl.DataFrame({'unit': pl.Series([1,2,3]).unit.with_unit("m")})

In [15]:
(df.select(unit_op=plu.mean("unit"))
.with_columns(
    equal=(
        pl.col("unit_op").struct.field("value")
        == polars_op(pl.col("unit").struct.field("value"))
    ).all()
))

ColumnNotFoundError: unit